# Metaphysical Reps

🏋️

An interface for poetic training.  It guides the user to metaphorically consider a key term (such as "truth" or "error") in light of a metaphorical vehicle (such as a "friar" or a "car"); it does this by generating the beginnings of lines that describe things that are typically possessed by the vehicle (e.g. the "sermon" of the "friar").

## Get Data

This is data that I generated from Project Gutenberg text (for another project and am repurposing here as best I can).

In [ ]:
!wget https://raw.githubusercontent.com/kbooten/ekphrasisgym/main/possessor2possessed_and_weights.json

--2023-05-26 15:55:20--  https://raw.githubusercontent.com/kbooten/ekphrasisgym/main/possessor2possessed_and_weights.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1521163 (1.5M) [text/plain]
Saving to: ‘possessor2possessed_and_weights.json.1’

possessor2possessed 100%[===================>]   1.45M  --.-KB/s    in 0.03s   

2023-05-26 15:55:20 (45.2 MB/s) - ‘possessor2possessed_and_weights.json.1’ saved [1521163/1521163]



In [ ]:
!wget https://raw.githubusercontent.com/kbooten/ekphrasisgym/main/noun2adjs_and_weights.json

--2023-05-26 15:55:20--  https://raw.githubusercontent.com/kbooten/ekphrasisgym/main/noun2adjs_and_weights.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7817280 (7.5M) [text/plain]
Saving to: ‘noun2adjs_and_weights.json.1’

noun2adjs_and_weigh 100%[===================>]   7.46M  --.-KB/s    in 0.04s   

2023-05-26 15:55:21 (170 MB/s) - ‘noun2adjs_and_weights.json.1’ saved [7817280/7817280]



In [ ]:
!wget https://raw.githubusercontent.com/kbooten/ekphrasisgym/main/noun2verbs_and_weights.json

--2023-05-26 15:55:21--  https://raw.githubusercontent.com/kbooten/ekphrasisgym/main/noun2verbs_and_weights.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3474424 (3.3M) [text/plain]
Saving to: ‘noun2verbs_and_weights.json.1’

noun2verbs_and_weig 100%[===================>]   3.31M  --.-KB/s    in 0.03s   

2023-05-26 15:55:21 (128 MB/s) - ‘noun2verbs_and_weights.json.1’ saved [3474424/3474424]



In [ ]:
import json

with open('possessor2possessed_and_weights.json','r') as f:
      possessor2possessed = json.load(f)

with open('noun2adjs_and_weights.json','r') as f:
      noun2adjs = json.load(f)

with open('noun2verbs_and_weights.json','r') as f:
      noun2verbs = json.load(f)

In [ ]:
possessors = list([k for k in possessor2possessed.keys() if len(possessor2possessed[k])>14])
len(possessors)

1110

In [ ]:
possessor2possessed['friar']

[['balsam', 99.03597122302159],
 ['cowl', 99.03597122302159],
 ['cord', 21.221993833504627],
 ['hood', 14.147995889003084],
 ['gown', 12.553855507143581],
 ['cell', 10.157535510053496],
 ['robe', 9.726747173689619],
 ['demise', 9.003270111183781],
 ['lantern', 7.502725092653149],
 ['habit', 7.48527689476326],
 ['frock', 7.282056707575116],
 ['preaching', 7.073997944501542],
 ['execution', 5.825645366060093],
 ['sermon', 2.8295991778006164],
 ['dress', 2.2255274432139682],
 ['appeal', 2.0211422698575836],
 ['cloak', 1.9546573267701628],
 ['servant', 1.4422714255779843],
 ['crown', 1.3754996003197444],
 ['thought', 0.9615142837186562],
 ['speech', 0.7282056707575116],
 ['death', 0.5018714757923392],
 ['words', 0.4058851279632032],
 ['face', 0.3571008577753182],
 ['mouth', 0.3368570449762639],
 ['hand', 0.2947499143542309],
 ['voice', 0.28901548022282564],
 ['eye', 0.21910613102438406],
 ['heart', 0.18339994670929924],
 ['eyes', 0.17601179719731325]]

## Vector Model

I'll use some Glove vectors to refine my selection of words.

In [ ]:
import gensim.downloader as api
vector_model = api.load("glove-wiki-gigaword-50")

In [ ]:
import inflect
inf = inflect.engine()

In [ ]:
inf.singular_noun("dog")

False

In [ ]:
def compare_words(w0,w1):
  if w0 in vector_model:
    if w1 in vector_model:
      return vector_model.similarity(w0,w1)
  return False

compare_words('dog','smile')

0.49237353

A function that prunes a list of words so that it does not contain overly-related words.

In [ ]:
def remove_too_close(list_of_words,too_close = .72):
  list_of_words_again = reversed(list_of_words)
  to_remove = []
  for w in list_of_words_again:
    for _w in list_of_words:
      if (_w not in to_remove and w!=_w):
        sim = compare_words(w,_w)
        if sim>=too_close:
          to_remove.append(w)
  for to_rem in list(set(to_remove)):
    list_of_words.remove(to_rem)
  return list_of_words


remove_too_close(['dog','dogs','hat','hats','porch'])

['dog', 'hat', 'porch']

In [ ]:
vector_model.similarity('hat','hats')

0.74306446

A function that reranks (and takes the top `n` of a list of) a list words according to how similar the words are to some `comparison` term.  

In [ ]:
def get_choices(list_of_words,comparison,choose_from_top=14):
  #all_possessed = [item for item,value in possessor2possessed[possessor]]
  list_of_words_scored = [(w,compare_words(comparison,w)) for w in list_of_words]
  list_of_words_scored.sort(key=lambda x: x[1], reverse=True)
  list_of_words_scored = [w for w,score in list_of_words_scored][:choose_from_top]
  list_of_words_scored = remove_too_close(list_of_words_scored)
  return list_of_words_scored

get_choices([w for w,score in possessor2possessed['lawyer']],'lawyer')

['clerk',
 'office',
 'firm',
 'letter',
 'clients',
 'brief',
 'fees',
 'distinction',
 'communication',
 'portfolio']

In [ ]:
import random

def random_sample_keep_order(a_list,k):
  a_sample = random.sample(a_list,k)
  a_sample_in_order = [i for i in a_list if i in a_sample]
  return a_sample_in_order

random_sample_keep_order([1,2,3,4,5,6],3)

[1, 2, 4]

In [ ]:
from nltk import corpus,download
download('stopwords')
stopwords = corpus.stopwords.words('english')
stopwords += ['own','very','make']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def is_it_singular(noun):
  if noun in noun2adjs:
    if noun2adjs[noun]['num']==str(1):
      return True
    else:
      return False
  elif noun in noun2verbs:
    if noun2verbs[noun]['num']==str(1):
      return True
    else:
      return False
  elif noun.endswith('ss'):
    return True
  else:
    if inf.singular_noun(noun)==False:
      return True
    else:
      return False

print(is_it_singular("ass"))
print(is_it_singular("asses"))
print(is_it_singular("progress"))
print(is_it_singular("virtue"))

True
False
True
True


## Main Function

In [ ]:
def do_some_reps(keyword,vehicle=None,max_reps=4,adj_prob=.3,verb_prob=.5):

  ## either use the vehicle supplied or choose one
  if vehicle == None:
    a_possessor = random.choice(possessors)
  else:
    a_possessor = vehicle

  ## make it singular if it isn't
  if is_it_singular(a_possessor)==False:
    a_possessor_singular = inf.singular_noun(a_possessor) ## make singular with inflect
    a_possessor = a_possessor_singular if a_possessor_singular!=False else a_possessor  ## this is just in case inf thinks it IS singular

  ## choose a certain number of possessed after reranking them
  possessed = [w for w,score in possessor2possessed[a_possessor]]
  choices = get_choices(possessed,a_possessor)
  possessed = random_sample_keep_order(choices,max_reps)

  ## format first string
  but_or_not = random.choice(["","","","but "])
  a_or_an = "an" if a_possessor[0] in "aeiou" else "a"
  print("\033[38;5;250m%s is %s%s %s\033[0;0m" % (keyword,but_or_not,a_or_an,a_possessor))

  ## loop through possessed
  for i in possessed:

    ## maybe add adjective
    if (random.random()<adj_prob or i not in noun2adjs):
      adj = ""
    else:
      adjs = [v for v,s in noun2adjs[i]['adjs'] if v not in stopwords]
      adj_choices = get_choices(adjs,a_possessor,choose_from_top=3)
      adj = random.choice(adj_choices)+" "

    ## format string and get user input
    is_or_are = "is" if is_it_singular(i) else "are"
    input_string = "\033[38;5;250m  and its %s%s %s \033[0;0m" % (adj,i,is_or_are)
    input(input_string)

    ## maybe add verb
    if (random.random()<verb_prob and i in noun2verbs):
      verbs = [v for v,s in noun2verbs[i]['verbs'] if v not in stopwords]
      verb_choices = get_choices(verbs,i,choose_from_top=6)
      a_verb = random.choice(verb_choices)
      this_or_these = "this" if is_it_singular(i) else "these"
      verb_framing = random.choice(["could","shall","will","will","could not","should","mustn't","does","does","does"])
      if (verb_framing == "does" and is_it_singular(i)==False):
        verb_framing = "do"
      input_string = "\033[38;5;250m    and %s %s %s %s \033[0;0m" % (this_or_these,i,verb_framing,a_verb)
      input(input_string)

#do_some_reps("history")

# Let's Get Pumped 💪

In [ ]:
items_to_consider = [
    "truth",
    "youth",
    "poesy",
    "fashion",
    "time",
    "money",
    "work",
    "error",
    "doubt",
    "thought",
    "history",
    "a flower",
    "tomorrow",
    "your face",
    "the night"
    "the day",
    "mercy",

]

In [ ]:
do_some_reps(random.choice(items_to_consider))

tomorrow is a star


***
